## Example from R1



In [ ]:
import requests

# Replace with your actual API keys
ALPHA_VANTAGE_API_KEY = 'your_alpha_vantage_api_key'
OLLAMA_API_KEY = 'your_ollama_api_key'
OLLAMA_API_URL = 'https://api.ollama.com/v1/chat/completions'

def get_stock_price(symbol):
    """Fetch the latest stock price using Alpha Vantage."""
    url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'
    response = requests.get(url)
    data = response.json()
    
    if 'Global Quote' in data:
        return data['Global Quote']['05. price']
    else:
        return None

def ask_ollama(prompt):
    """Send a prompt to the Ollama API and get the response."""
    headers = {
        'Authorization': f'Bearer {OLLAMA_API_KEY}',
        'Content-Type': 'application/json',
    }
    payload = {
        'model'="llama3.1:8b",  # or any other model supported by Ollama
        'messages': [{'role': 'user', 'content': prompt}],
        'max_tokens': 150,
    }
    response = requests.post(OLLAMA_API_URL, headers=headers, json=payload)
    data = response.json()
    
    return data['choices'][0]['message']['content']

def stock_price_agent(user_query):
    """Agent that processes the user query and returns the stock price."""
    # Extract the stock symbol from the user query using Ollama
    prompt = f"Extract the stock symbol from the following query: '{user_query}'. Return just the symbol."
    symbol = ask_ollama(prompt).strip()
    
    if symbol:
        price = get_stock_price(symbol)
        if price:
            return f"The current price of {symbol} is ${price}."
        else:
            return f"Could not fetch the price for {symbol}."
    else:
        return "Could not identify the stock symbol in your query."

# Example usage
user_query = "What's the stock price of Apple?"
response = stock_price_agent(user_query)
print(response)

## How to use LangChain and GPT to ask question on multiple pdf documents

https://www.datascienceengineer.com/blog/post-multiple-pdfs-with-gpt

! pip install langchain openai pypdf chroma



In [8]:
import sys
import os
import copy
    
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os

In [10]:
os.environ["OPENAI_API_KEY"] = "your_openAI_key"

In [ ]:
loader = PyPDFLoader("appl-20230930.pdf")

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
pages = loader.load_and_split(text_splitter)

In [ ]:
directory = 'index_store'
vector_index = Chroma.from_documents(pages, OpenAIEmbeddings(), persist_directory=directory)
vector_index.persist() # actually the Chroma client automatically persists the indexes when it is disposed - however better save then sorry :-)

In [ ]:
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k":6})
# create the chain for allowing us to chat with the document
qa_interface = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [12]:
query = """
You are an expert financial analyst. Please estimate what are the following values:

1. Working Capital
2. Total Assets
3. Retained Earnings
4. Earnings Before Interest and Taxes (EBIT)
5. Market Value of Equity
6. Total Liabilities
7. Sales

Please present the metrics in a structured format:
- Working Capital: [value]
- Total Assets: [value]
- Retained Earnings: [value]
- EBIT: [value]
- Market Value of Equity: [value]
- Total Liabilities: [value]
- Sales: [value]
"""

res = qa_interface(query)
res['result']

In [13]:
query = """
You are an expert financial analyst. 
Summarise the balance sheet.
"""

res = qa_interface(query)
res['result']

In [14]:
query = """
What is the working capital
"""

res = qa_interface(query)
res['result']



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Bitcoin initial release date'}`


Page: Silk Road (marketplace)
The initial release date of Bitcoin is October 31, 2008.m

> Finished chain.


In [17]:
response = agent_executor.invoke({"input": "what are the prices for AAPL stock between 2023-01-01 and 2023-02-01?"})



> Entering new AgentExecutor chain...

Invoking: `get_stock_prices_and_volumes` with `{'end_date': '2023-02-01', 'start_date': '2023-01-01', 'symbol': 'AAPL'}`


Error fetching stock prices and volumes for AAPL: No module named 'numpy.rec'I'm unable to find the requested data. However, I can suggest some alternatives to get the information you need.

You can use a financial API such as Alpha Vantage or Yahoo Finance to retrieve historical stock prices. These APIs provide a simple way to access and parse stock price data.

Here's an example of how you could use the Alpha Vantage API:

```
import requests

api_key = "YOUR_API_KEY"
symbol = "AAPL"

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    prices = data["Time Series (Daily)"]
    
    for date, price in prices.items():
        print(f"{date}: {price['4. close']}")
else:
    print("Fai

## Exmple using R1 queries

To create a Simple RAG (Retrieval-Augmented Generation) agent in Python that reads SEC filings (PDFs) and generates a recommendation with 7 important KPIs, you can follow these steps:

    - Extract Text from PDFs : Use a library like PyPDF2 or PyMuPDF (fitz) to extract text from the PDF files.
    - Process the Text : Use a language model (like OpenAI GPT) to analyze the text and extract key financial metrics (KPIs).
    - Generate Recommendations : Use the LLM to generate a financial analyst-style recommendation based on the extracted KPIs.



In [ ]:
import fitz  # PyMuPDF for PDF text extraction
import openai  # OpenAI API for RAG
import os

# Replace with your OpenAI API key
OPENAI_API_KEY = "your_openai_api_key"
openai.api_key = OPENAI_API_KEY

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_kpis_and_recommendation(text):
    """Analyze the extracted text using OpenAI GPT to extract KPIs and generate a recommendation."""
    prompt = f"""You are a financial analyst. Analyze the following text from an SEC filing and extract 7 key performance indicators (KPIs) that are critical for evaluating the company's financial health. Also, provide a short recommendation based on these KPIs.

Text:
{text}

Format your response as follows:
KPIs:
1. [KPI 1]
2. [KPI 2]
3. [KPI 3]
4. [KPI 4]
5. [KPI 5]
6. [KPI 6]
7. [KPI 7]

Recommendation:
[Your recommendation here]"""

    response = openai.ChatCompletion.create(
        model="gpt-4",  # or gpt-3.5-turbo
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
    )
    return response["choices"][0]["message"]["content"]

def process_sec_filing(pdf_path):
    """Process a single SEC filing PDF and generate KPIs and recommendation."""
    print(f"Processing SEC filing: {pdf_path}")
    text = extract_text_from_pdf(pdf_path)
    if not text:
        return "No text could be extracted from the PDF."
    
    result = extract_kpis_and_recommendation(text)
    return result

def main():
    # Path to the directory containing SEC filing PDFs
    pdf_directory = "sec_filings"  # Replace with your directory path
    if not os.path.exists(pdf_directory):
        print(f"Directory '{pdf_directory}' does not exist.")
        return
    
    # Process all PDFs in the directory
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            result = process_sec_filing(pdf_path)
            print(result)
            print("-" * 80)

if __name__ == "__main__":
    main()

Processing SEC filing: sec_filings/apple_10k_2023.pdf
KPIs:
1. Revenue Growth: 8% year-over-year
2. Net Income Margin: 25%
3. Earnings Per Share (EPS): $6.10
4. Operating Cash Flow: $100 billion
5. Debt-to-Equity Ratio: 1.2
6. Return on Equity (ROE): 35%
7. Current Ratio: 1.5

Recommendation:
Apple has shown strong financial performance with significant revenue growth and high profitability. The company’s robust operating cash flow and healthy ROE indicate efficient management. However, the debt-to-equity ratio suggests moderate leverage, which should be monitored. Overall, Apple is a solid investment with potential for continued growth.
--------------------------------------------------------------------------------

### The Altman Z-Score 

It is a financial formula used to predict the likelihood of a company going bankrupt within two years. It combines five financial ratios into a single score. 

The formula is:

Z=1.2⋅ 
Total Assets /
Working Capital
​	
 +1.4⋅ 
Total Assets /
Retained Earnings
​	
 +3.3⋅ 
Total Assets /
EBIT
​	
 +0.6⋅ 
Total Liabilities /
Market Value of Equity
​	
 +1.0⋅ 
Total Assets /
Sales
​	
 
Here’s how to calculate the Altman Z-Score in Python:

In [ ]:
def calculate_altman_z_score(working_capital, retained_earnings, ebit, total_assets, total_liabilities, market_value_of_equity, sales):
    """
    Calculate the Altman Z-Score for a company.

    Parameters:
    - working_capital: Working Capital (Current Assets - Current Liabilities)
    - retained_earnings: Retained Earnings
    - ebit: Earnings Before Interest and Taxes (EBIT)
    - total_assets: Total Assets
    - total_liabilities: Total Liabilities
    - market_value_of_equity: Market Value of Equity
    - sales: Sales or Revenue

    Returns:
    - Z-score: Altman Z-Score
    """
    # Calculate the five components of the Altman Z-Score
    x1 = working_capital / total_assets
    x2 = retained_earnings / total_assets
    x3 = ebit / total_assets
    x4 = market_value_of_equity / total_liabilities
    x5 = sales / total_assets

    # Calculate the Z-score
    z_score = 1.2 * x1 + 1.4 * x2 + 3.3 * x3 + 0.6 * x4 + 1.0 * x5

    return z_score

def interpret_altman_z_score(z_score):
    """
    Interpret the Altman Z-Score.

    Parameters:
    - z_score: Calculated Altman Z-Score

    Returns:
    - Interpretation: Interpretation of the Z-Score
    """
    if z_score > 2.99:
        return "Safe Zone: The company is financially stable with a low risk of bankruptcy."
    elif 1.81 <= z_score <= 2.99:
        return "Grey Zone: The company is at moderate risk of bankruptcy."
    else:
        return "Distress Zone: The company is at high risk

### To calculate the Altman Z-Score using data from an SEC 10-K filing.

we need to extract specific financial metrics from the filing. 

These metrics include:

    - Working Capital (Current Assets - Current Liabilities)
    - Retained Earnings
    - Earnings Before Interest and Taxes (EBIT)
    - Total Assets
    - Market Value of Equity (Market Capitalization)
    - Total Liabilities
    - Sales (Revenue)

In Python, we assume the SEC 10-K filing data is available in a structured format (e.g., extracted from the text or from a financial API). Here’s how to calculate the Altman Z-Score using this data:

In [ ]:
def calculate_altman_z_score(working_capital, retained_earnings, ebit, total_assets, market_value_equity, total_liabilities, sales):
    """
    Calculate the Altman Z-Score for a company.

    Parameters:
        working_capital (float): Working Capital (Current Assets - Current Liabilities)
        retained_earnings (float): Retained Earnings
        ebit (float): Earnings Before Interest and Taxes (EBIT)
        total_assets (float): Total Assets
        market_value_equity (float): Market Value of Equity (Market Capitalization)
        total_liabilities (float): Total Liabilities
        sales (float): Sales (Revenue)

    Returns:
        float: Altman Z-Score
    """
    # Calculate the 5 components of the Altman Z-Score
    x1 = working_capital / total_assets
    x2 = retained_earnings / total_assets
    x3 = ebit / total_assets
    x4 = market_value_equity / total_liabilities
    x5 = sales / total_assets

    # Calculate the Altman Z-Score
    z_score = 1.2 * x1 + 1.4 * x2 + 3.3 * x3 + 0.6 * x4 + 1.0 * x5
    return z_score

def interpret_altman_z_score(z_score):
    """
    Interpret the Altman Z-Score.

    Parameters:
        z_score (float): Calculated Altman Z-Score

    Returns:
        str: Interpretation of the score
    """
    if z_score > 2.99:
        return "Safe Zone: Low probability of bankruptcy."
    elif 1.81 <= z_score <= 2.99:
        return "Grey Zone: Moderate risk of bankruptcy."
    else:
        return "Distress Zone: High probability of bankruptcy."

def extract_financial_data_from_10k(sec_10k_data):
    """
    Extract necessary financial data from SEC 10-K filing (assume data is structured).
    
    Parameters:
        sec_10k_data (dict): A dictionary containing financial data from the 10-K filing.

    Returns:
        tuple: Financial metrics required for the Altman Z-Score calculation.
    """
    working_capital = sec_10k_data.get("current_assets", 0) - sec_10k_data.get("current_liabilities", 0)
    retained_earnings = sec_10k_data.get("retained_earnings", 0)
    ebit = sec_10k_data.get("ebit", 0)
    total_assets = sec_10k_data.get("total_assets", 0)
    market_value_equity = sec_10k_data.get("market_capitalization", 0)
    total_liabilities = sec_10k_data.get("total_liabilities", 0)
    sales = sec_10k_data.get("revenue", 0)

    return working_capital, retained_earnings, ebit, total_assets, market_value_equity, total_liabilities, sales

def main():
    # Example: Extracted financial data from SEC 10-K filing (replace with actual data)
    sec_10k_data = {
        "current_assets": 150000,  # in thousands
        "current_liabilities": 100000,  # in thousands
        "retained_earnings": 50000,  # in thousands
        "ebit": 30000,  # in thousands
        "total_assets": 200000,  # in thousands
        "market_capitalization": 250000,  # in thousands
        "total_liabilities": 120000,  # in thousands
        "revenue": 180000,  # in thousands
    }

    # Extract financial data
    working_capital, retained_earnings, ebit, total_assets, market_value_equity, total_liabilities, sales = \
        extract_financial_data_from_10k(sec_10k_data)

    # Calculate Altman Z-Score
    z_score = calculate_altman_z_score(working_capital, retained_earnings, ebit, total_assets, market_value_equity, total_liabilities, sales)

    # Interpret the Z-Score
    interpretation = interpret_altman_z_score(z_score)

    # Output results
    print(f"Altman Z-Score: {z_score:.2f}")
    print(f"Interpretation: {interpretation}")

if __name__ == "__main__":
    main()